In [2]:

import pandas as pd 
import yfinance as yf 


In [68]:
prices = yf.download(
    ["DX=F", "ES=F", "GC=F", "ZN=F"],
    "2018-01-01"

)["Close"]
prices

[*********************100%***********************]  4 of 4 completed


Ticker,DX=F,ES=F,GC=F,ZN=F
Date,,,,
2018-01-02,91.572998,2693.00,1313.699951,123.687500
2018-01-03,91.896004,2711.00,1316.199951,123.781250
2018-01-04,91.581001,2723.75,1319.400024,123.671875
2018-01-05,91.672997,2742.50,1320.300049,123.515625
2018-01-08,92.099998,2746.75,1318.599976,123.484375
...,...,...,...,...
2025-02-19,107.063004,6163.00,2919.399902,108.953125
2025-02-20,106.268997,6136.50,2940.000000,109.171875
2025-02-21,106.518997,6029.00,2937.600098,109.687500


In [69]:
returns = prices.pct_change().dropna()
returns

/tmp/ipykernel_1913/2165859093.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


Ticker,DX=F,ES=F,GC=F,ZN=F
Date,,,,
2018-01-03,0.003527,0.006684,0.001903,0.000758
2018-01-04,-0.003428,0.004703,0.002431,-0.000884
2018-01-05,0.001005,0.006884,0.000682,-0.001263
2018-01-08,0.004658,0.001550,-0.001288,-0.000253
2018-01-09,0.001737,0.002002,-0.005233,-0.003543
...,...,...,...,...
2025-02-19,0.001010,0.002644,-0.004162,0.001005
2025-02-20,-0.007416,-0.004300,0.007056,0.002008
2025-02-21,0.002353,-0.017518,-0.000816,0.004723


In [70]:
returns.index

DatetimeIndex(['2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
               '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12',
               '2018-01-16', '2018-01-17',
               ...
               '2025-02-12', '2025-02-13', '2025-02-14', '2025-02-17',
               '2025-02-18', '2025-02-19', '2025-02-20', '2025-02-21',
               '2025-02-24', '2025-02-25'],
              dtype='datetime64[ns]', name='Date', length=1799, freq=None)

In [71]:
# convert to monthly returns 
returns_monthly  = (1+ returns).resample("ME").prod() -1 
returns_monthly

Ticker,DX=F,ES=F,GC=F,ZN=F
Date,,,,
2018-01-31,-0.028600,0.049294,0.019259,-0.017054
2018-02-28,0.017953,-0.039370,-0.017550,-0.007583
2018-03-31,-0.008161,-0.026340,0.005549,0.004015
2018-04-30,0.020242,0.001513,-0.004989,-0.012511
2018-05-31,0.025417,0.022100,-0.012232,0.009274
...,...,...,...,...
2024-10-31,0.033396,-0.013028,0.038769,-0.033361
2024-11-30,0.018772,0.054544,-0.029690,0.004950
2024-12-31,0.023321,-0.019127,-0.010463,-0.020408


# Calculating Risk-Parity Asset Weights 
- Idea: assign weights to each of the stocks/assets in the portfolio by the volotility they have
- More volatility for asset `i` means a smaller weight 
- use the inverse volatiliy: $1/\sigma_i $

In [72]:
# volatility over the last 36 months/3years
rolling_vol = returns_monthly.rolling(36).std()
rolling_vol.shape

(86, 4)

In [73]:
#rolling_vol = rolling_vol.dropna()
#rolling_vol

In [74]:
rolling_inverse_vol = 1/rolling_vol

In [75]:
# compute the weigths of the assets 
# for each row, we have a set of weigths that sum to 1 
risk_parity_weights = rolling_inverse_vol.apply(lambda x: x/x.sum(), axis = 1 )
risk_parity_weights = risk_parity_weights.dropna()


In [76]:
#check each row sum to 1 
risk_parity_weights.sum(axis = 1).sum()  == float(risk_parity_weights.shape[0])

np.True_

In [77]:
risk_parity_weights.shape

(51, 4)

In [78]:
float(risk_parity_weights.shape[0])

51.0

## Adjust weights to Avoid Look Ahead Bias

In [79]:
# shift by 1 day 
shifted_weights = risk_parity_weights.shift(1)
shifted_weights

Ticker,DX=F,ES=F,GC=F,ZN=F
Date,,,,
2020-12-31,NaN,NaN,NaN,NaN
2021-01-31,0.327354,0.096373,0.148576,0.427697
2021-02-28,0.333472,0.094457,0.143110,0.428962
2021-03-31,0.347584,0.097678,0.139500,0.415238
2021-04-30,0.346947,0.101076,0.143576,0.408401
2021-05-31,0.344039,0.100049,0.142577,0.413335
2021-06-30,0.352041,0.099462,0.136599,0.411898
2021-07-31,0.343568,0.101399,0.134343,0.420689
2021-08-31,0.343630,0.101617,0.135493,0.419260


In [80]:
risk_parity_weights

Ticker,DX=F,ES=F,GC=F,ZN=F
Date,,,,
2020-12-31,0.327354,0.096373,0.148576,0.427697
2021-01-31,0.333472,0.094457,0.143110,0.428962
2021-02-28,0.347584,0.097678,0.139500,0.415238
2021-03-31,0.346947,0.101076,0.143576,0.408401
2021-04-30,0.344039,0.100049,0.142577,0.413335
2021-05-31,0.352041,0.099462,0.136599,0.411898
2021-06-30,0.343568,0.101399,0.134343,0.420689
2021-07-31,0.343630,0.101617,0.135493,0.419260
2021-08-31,0.344138,0.101467,0.136297,0.418097


In [81]:
# create a portfolio from the risk parity weigths and 
# compute the portfolio return, which is the wi * returni 
weighted_returns = returns_monthly*risk_parity_weights

In [82]:
portfolio_returns = weighted_returns.sum(axis = 1)
portfolio_returns

Date
2018-01-31    0.000000
2018-02-28    0.000000
2018-03-31    0.000000
2018-04-30    0.000000
2018-05-31    0.000000
                ...   
2024-10-31    0.005094
2024-11-30    0.010006
2024-12-31   -0.004068
2025-01-31    0.016198
2025-02-28    0.004046
Freq: ME, Length: 86, dtype: float64

In [85]:
print(f"Portfolio return: {((1 + portfolio_returns).prod()-1)*100 :.4f}%")

Portfolio return: 13.4886%
